In [13]:
from collections import Counter
import pymongo
import pickle

In [7]:
client = pymongo.MongoClient("mongodb://localhost:37017" )
db=client["lol"]
coll=db["match_data"]

In [1]:
def pull_champions_from_game_data(mongo_object):
    """Takes a dictionary stored in my mongodb.
    Returns ordered lists of allied champions,
    enemy champions, and the winning team id as an int.
    """
    champs=[]
    winning_team=0
    for player in mongo_object["data"]["participants"]:
        champs.append(player["championId"])
    allied_champions=champs[:5]
    enemy_champions=champs[5:]
    if mongo_object["data"]["teams"][0]["win"]=="Fail":
        winning_team=100
    else:
        winning_team=200
    return allied_champions,enemy_champions,winning_team
        

In [17]:
def build_initial_winrate_dict(iterable_mongo):
    champ_plays=Counter()
    champ_wins=Counter()
    for game in iterable_mongo:
        allies,enemies,win=pull_champions_from_game_data(game)
        for champ in allies:
            champ_plays[champ]+=1
            if win==100:
                champ_wins[champ]+=1
        for champ in enemies:
            champ_plays[champ]+=1
            if win==200:
                champ_wins[champ]+=1
    return champ_wins,champ_plays
    

In [24]:
curs=coll.find().limit(6000)

In [25]:
wins,plays = build_initial_winrate_dict(curs)

In [11]:
hmm=defaultdict()

In [26]:
win_percent={}
for x in wins.keys():
    win_percent[x]=wins[x]/float(plays[x])

In [30]:
sorted(win_percent.items(),key=lambda(x): x[1], reverse=True)

[(69, 0.6057142857142858),
 (26, 0.5862068965517241),
 (240, 0.5804597701149425),
 (107, 0.5755813953488372),
 (127, 0.5728155339805825),
 (85, 0.5721649484536082),
 (60, 0.5714285714285714),
 (223, 0.5707547169811321),
 (6, 0.570281124497992),
 (68, 0.5699481865284974),
 (41, 0.5668449197860963),
 (134, 0.5665914221218962),
 (76, 0.5656108597285068),
 (77, 0.5637583892617449),
 (13, 0.5622641509433962),
 (15, 0.5606557377049181),
 (28, 0.558792924037461),
 (7, 0.5545851528384279),
 (432, 0.5529411764705883),
 (516, 0.5520581113801453),
 (30, 0.5490196078431373),
 (429, 0.5463258785942492),
 (412, 0.545543584720862),
 (79, 0.5424528301886793),
 (14, 0.5415384615384615),
 (268, 0.5406162464985994),
 (236, 0.5404580152671755),
 (164, 0.5388888888888889),
 (154, 0.5375),
 (35, 0.5366795366795367),
 (163, 0.5365853658536586),
 (136, 0.5346534653465347),
 (64, 0.5302826379542396),
 (38, 0.5298507462686567),
 (3, 0.5257452574525745),
 (420, 0.5256410256410257),
 (497, 0.5253772290809328),
 (